## Weight Window (wwinp) files and GVRs

```{eval-rst}
The complete API can be found at :py:class:`f4enix.output.ww_gvr.WW`
```

The ww_gvr module can operate on Weight Window files to, for example, reduce the likelihood of long history problems, or to add or remove a secondary particle.
It can also generate Global Variance Reduction files (GVR) by reading a Meshtally file.
This module can be used via Python scripting or through a command line interface.

### Example of use via Python scripting

In [10]:
from f4enix.input.ww_gvr import WW

# Load the weight window by reading a WW file
ww = WW.load_from_ww_file("ww_simple_cart")

print(ww)

 ww_simple_cart weight window
       ----From---- -----To----- --No. Bins--
 I -->    -15.0         15.0          2      
 J -->    -15.0         16.0          3      
 K -->    -15.0         20.0          1      

 Cartesian coordinates, 6 voxels.

 The weight window contains 1 particle/s

 ---------------------Neutron----------------------
 Energy bins:         [100.0]
 Min value:           8.98E-02
 Max value:           6.73E-01
 No.Bins > 0 [%]:     100.0%
 Average ratio:       5.97E+00
 Max ratio:           6.74E+00


In [11]:
import os
import tempfile
from pathlib import Path

# Normalize the weight window
ww.multiply(1.2)

# Soft the weight window
ww.soften(0.5)

# Add a secondary particle
ww.add_particle(norm=0.2, soft=1.)

# Ensure that there are no high ratios between voxels to avoid long histories
ww.mitigate_long_histories(max_ratio=10.)

with tempfile.TemporaryDirectory() as outdirname:
    outdir = Path(outdirname)

    # Save the modified ww to a new file as outfile:
    ww.write_to_ww_file(os.path.join(outdir, "mod_ww"))
    # Export the ww as a VTK to visualize it in Paraview
    ww.export_as_vtk(os.path.join(outdir, "mod_ww.vts"))

print(ww)

 ww_simple_cart weight window
       ----From---- -----To----- --No. Bins--
 I -->    -15.0         15.0          2      
 J -->    -15.0         16.0          3      
 K -->    -15.0         20.0          1      

 Cartesian coordinates, 6 voxels.

 The weight window contains 2 particle/s

 ---------------------Neutron----------------------
 Energy bins:         [100.0]
 Min value:           3.28E-01
 Max value:           8.99E-01
 No.Bins > 0 [%]:     100.0%
 Average ratio:       2.44E+00
 Max ratio:           2.60E+00

 ----------------------Photon----------------------
 Energy bins:         [100.0]
 Min value:           6.57E-02
 Max value:           1.80E-01
 No.Bins > 0 [%]:     100.0%
 Average ratio:       2.44E+00
 Max ratio:           2.60E+00


In [12]:
# Uncomment for an interactive plot
# ww.plot()

### Creation of a GVR

A GVR (Global Variance Reduction) is a type of WW that aims to normalize the neutron population inside a region. F4Enix can generate automatically a GVR from the neutron flux results recorded in a FMESH file. 

The convergence of the FMESH results can be challenging wihtout a GVR. When creating a GVR for the first time, an iterative approach is proposed:
1. Run a simulation with a DCF < 1.0 applied to all the cells. This will allow an easier neutron penetration.
2. Generate a GVR with the FMESH obtained in the previous step.
3. Run a simulation without DCFs but applying the GVR obtained in the previous step.
4. Return to step 2 until the GVR has the sufficient quality.

In [13]:
gvr = WW.create_gvr_from_meshtally_file(
    "meshtally_cart", 
    maximum_splitting_ratio=5.0, 
    softening_factor=1.0
    )
# Ensure that there are no high ratios between voxels to avoid long histories
gvr.mitigate_long_histories(max_ratio=10.)
# Add photon WW values to the GVR
gvr.add_particle(norm=0.2, soft=1.)
print(gvr)

 meshtally_cart weight window
       ----From---- -----To----- --No. Bins--
 I -->    -10.0         20.0          3      
 J -->     20.0         40.0          4      
 K -->     15.0         25.0          2      

 Cartesian coordinates, 24 voxels.

 The weight window contains 2 particle/s

 ---------------------Neutron----------------------
 Energy bins:         [100.0]
 Min value:           1.30E-01
 Max value:           3.33E-01
 No.Bins > 0 [%]:     100.0%
 Average ratio:       1.25E+00
 Max ratio:           1.31E+00

 ----------------------Photon----------------------
 Energy bins:         [100.0]
 Min value:           2.59E-02
 Max value:           6.67E-02
 No.Bins > 0 [%]:     100.0%
 Average ratio:       1.25E+00
 Max ratio:           1.31E+00


### Use via a Command Line Interface
The CLI can be invoked by `python -m f4enix.input.ww_gvr` 


### Theory
#### How the mitigate_long_histories method works
The weigth window voxels with a ratio higher than max_ratio will be set to zero. This stops the particle from over-splitting by removing the WW from the most problematic areas. The ratio of a voxel X is calculated as ${max}(V_x/V_i)$ where $V_x$ is the WW value of the voxel and $V_i$ are the WW values of the voxels neighboring voxel X.  

#### Generation of a GVR
A GVR is generated using a FMESH file as input.The selected MESHTAL is loaded and processed as follows:

$V_i = (\frac{\phi_i}{{max}(\phi)} \cdot \frac{2}{\beta + 1})^S$

Where:
* $V_i$ is the WW value of the voxel i
* $\phi_i$ is the neutron flux value at voxel i
* ${max}(\phi)$ is the maximum neutron flux value found in the whole MESHTAL
* $\beta$ is the mazimum splitting ratio (default 5)
* $S$ is the softening factor to mitigate the splitting (default 1)

References:
1. A.J. van Wijk, G. Van den Eynde, J.E. Hoogenboom, An easy to implement global variance reduction procedure for MCNP, Annals of Nuclear Energy, Volume 38, Issue 11, 2011, Pages 2496-2503, ISSN 0306-4549, https://doi.org/10.1016/j.anucene.2011.07.037.
2. Andrew Davis, Andrew Turner, Comparison of global variance reduction techniques for Monte Carlo radiation transport simulations of ITER, Fusion Engineering and Design, Volume 86, Issues 9–11, 2011, Pages 2698-2700, ISSN 0920-3796, https://doi.org/10.1016/j.fusengdes.2011.01.059.

#### Verification process
The module has been verified in a first step employing simple weight window meshes and verifying each single function. The WW values were manually input to easily check the results from typical operations. The reader and the plot function was endorsed double checking all possible sections. As these files are relatively small, the complete WW were double checked visually, parameter by parameter.